<a href="https://colab.research.google.com/github/luisanovaes/GitHub/blob/master/Classificador_BERT_Portugues.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install simpletransformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

Mounted at /content/drive


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

##importando a tabela
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1eS2a9h676hpFhQsisSdJCgdqYaDpReyMgL-eREZ4-XQ/edit#gid=1423440435')

#Selecionando os dados de cada aba
sheet1 = wb.worksheet('Treino')
sheet2 = wb.worksheet('Teste')

#Selecionando os dados e passando para um dataframe
data1 = sheet1.get_all_values()
data2 = sheet2.get_all_values()

train_df = pd.DataFrame(data1)
train_df.columns = ['query','classificacao']
train_df['classificacao'] = train_df['classificacao'].astype(int)


test_df = pd.DataFrame(data2)
test_df.columns = ['query','classificacao']
test_df['classificacao'] = test_df['classificacao'].astype(int)


#vizualizar dados
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 361 entries, 0 to 360
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   query          361 non-null    object
 1   classificacao  361 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 5.8+ KB


In [ ]:
# Classificador

from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Preparing train data
#train_data = train_df

# Preparing eval data
#eval_data = test_df


# Optional model configuration
model_args = ClassificationArgs(num_train_epochs=1)
model_args.overwrite_output_dir = True

# Create a ClassificationModel
model = ClassificationModel(
    'bert',
    'neuralmind/bert-base-portuguese-cased',
    num_labels=3,
    use_cuda=False,
    args=model_args,   
) 

# Train the model
model.train_model(train_df)
model_args.overwrite_output_dir =True


# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test_df)




In [ ]:
# Make predictions with the model
predictions, raw_outputs = model.predict(["Petição Inicial"])

print(result,'\n')
print(model_outputs, '\n')
print(predictions, '\n', raw_outputs)
print(wrong_predictions)


df_output = pd.DataFrame(model_outputs)
df_wrong_predictions = pd.DataFrame(wrong_predictions)

df_output.to_csv('drive/MyDrive/Results-BERT/output.csv')




INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

{'mcc': 0.7770917790412966, 'eval_loss': 0.2901420544569745} 

[[-1.86752677  3.64318037 -2.2452302 ]
 [-1.87908125  3.58809352 -2.16652489]
 [-2.04709339  3.85009742 -2.27247024]
 ...
 [ 1.44130814 -2.36633968  1.19212961]
 [ 1.38683045 -3.41124129  2.42682314]
 [ 2.39203858 -2.52644348  0.47537506]] 

[2] 
 [[-0.32236293 -2.96249866  3.38550162]]
[]
